<a href="https://colab.research.google.com/github/BNIA/VitalSigns/blob/main/FDIC_Banks_Create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install -U -q PyDrive
! pip install geopy
! pip install geopandas
! pip install geoplot

In [ ]:
!apt install libspatialindex-dev
!pip install rtree

In [ ]:
%%capture
!pip install dataplay

In [ ]:
%%capture 
# These imports will handle everything
import os
import sys
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import psycopg2
import pyproj
from pyproj import Proj, transform
# conda install -c conda-forge proj4
from shapely.geometry import Point
from shapely import wkb
from shapely.wkt import loads
# https://pypi.org/project/geopy/
from geopy.geocoders import Nominatim

# In case file is KML, enable support
import fiona
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

## CSA and Baltimore

In [ ]:
#collapse_output
#collapse_input
csa = "https://services1.arcgis.com/mVFRs7NF4iFitgbY/ArcGIS/rest/services/Tpop/FeatureServer/1/query?where=1%3D1&outFields=*&returnGeometry=true&f=pgeojson"
csa = gpd.read_file(csa);
csa.head(1)

In [ ]:
url2 = "https://services1.arcgis.com/mVFRs7NF4iFitgbY/ArcGIS/rest/services/Tpop/FeatureServer/0/query?where=1%3D1&outFields=*&returnGeometry=true&f=pgeojson"
csa2 = gpd.read_file(url2);
csa2['CSA2010'] = csa2['City_1'] 
csa2['OBJECTID'] = 56 
csa2 = csa2.drop(columns=['City_1'])
csa2.head()

Append Baltimore City to the top of the CSA list. You use concat and not append to do this. We put it on the top of the df because when performing the ponp it returns only the last matching columns CSA Label. 

In [ ]:
# csa = pd.concat([csa2, csa], ignore_index=True)
csa = csa.append(csa2).reset_index(drop=True)

In [ ]:
csa.head(2)

In [ ]:
csa.tail(2)

# Read in Banks

In [ ]:
banks = gpd.read_file('BCityBanks_2019_CSACity.shp')
banks = banks.fillna('Baltimore City')
banks['count'] = 1
banks= banks[['CSA','count']]
banks = banks.groupby('CSA').sum(numeric_only=True) 
# Make sure ALL csas and BaltimoreCity are included. among other things
banks = csa[ ['CSA2010','tpop10'] ].merge( banks, left_on='CSA2010', right_on='CSA', how='outer' ) 
# Update the baltimore CSA.
banks.at[55,'count'] = banks['count'].sum()
banks.head(3)

In [ ]:

banks.tail(3)

In [ ]:
# Create the Indicator
banks['banks19'] = banks['count'] * 1000 / banks['tpop10'] 

banks.to_csv('149-banks-19.csv', index=False)

banks.head(68)
